# Example

This minimal example introduces the basic workflow for using the Adaptive Stratified Sampling package and showcases its features.

In [ ]:
import numpy as np

# Import the modules, note that the package already needs to be installed
import adaptive_stratification as adss
from adaptive_stratification.stratification import AdaptiveStratification as estimator

## Problem set-up

We begin by defining a test function. As an example, here we use the function

$f(\xi_1,\xi_2, \dots, \xi_d) = I\left(\sum_{i=1}^d\xi_i^2\le r^2\right)$

with $r = \sqrt{2/\pi}$. For $d=2$, this value of $r$ yields a true expected value of $0.5$.

Note that this particular test function can be used flexibly for different dimensions. The actual dimension $d$ for the input needs to be set during the instantiation of the 'solver' below.

Recall that the function $f$ needs to be defined on the $d$-dimensional unit cube. Depending on your problem, it may thus be necessary to transform the problem formulation. This is not needed here, of course.

In [ ]:
def test_fun(xi: np.ndarray):
    # xi contains sampling points within the d-dimensional unit cube. 
    return np.sum(xi ** 2, axis=1) <= 0.7978845608028654 ** 2  

Next, we define the sampling procedure's settings, including the problem dimension $d$.

In [ ]:
alpha = 0.9  # alpha: proportion of samples allocated optimally
N_max = int(1e4)  # numbers of max samples used
SR_const = 30  # increase per adaptation iteration
dimension = 2 # stochastic dimension

### Hyperrectangular tessellation
Now, we can run the Adaptive Stratified Sampling routine. First, we use an adaptive stratification based on hyperrectangles.

In [ ]:
est_hyper = estimator(test_fun, dimension, N_max, SR_const, alpha, type_strat='hyperrect', rand_gen=None, dynamic=False)
QoI, ignore, N_strat, QoI_var = est_hyper.solve()
print(f'After splitting the domain into {N_strat} strata, we found that the quantity of interest is {QoI} with an estimated variance of {QoI_var}')

### Simplex tessellation
Next, we repeat the same experiment using adaptive stratification based on simplices.

In [ ]:
est_simp = estimator(test_fun, dimension, N_max, SR_const, alpha, type_strat='simplex', rand_gen=None, dynamic=False)
QoI, ignore, N_strat, QoI_var = est_simp.solve()
print(f'After splitting the domain into {N_strat} strata, we found that the quantity of interest is {QoI} with an estimated variance of {QoI_var}')

## Visualization

For the cases where the output is either one or two-dimensional, you can use the visualization feature provided in the sub-package to either show a picture depicting the stratification in each step or at the end.

### Show the final stratification
For example, to show the final stratification, we can use the following.

In [ ]:
from adaptive_stratification.visualization.plot_stratification import AdaptiveStratificationVisualization as AdaptiveStratification

estimator_result = AdaptiveStratification(test_fun, dimension, N_max, SR_const, alpha, dynamic=False, type_strat='hyperrect', rand_gen=None)
QoI, ignore, N_strat, QoI_var = estimator_result.solve_vis_result()

### Show current stratification at each step
Similarly, to show the current stratification at each step as the adaptation progresses, we use:

In [ ]:
estimator_steps = AdaptiveStratification(test_fun, dimension, N_max, SR_const, alpha, dynamic=False, type_strat='hyperrect', rand_gen=None)
QoI, ignore, N_strat, QoI_var = estimator_steps.solve_vis_steps()

## Logging
Finally, the Adaptive Stratified Sampling package comes with a logging functionality, which can be used as shown below. There we use a one-dimensional test function, where we already obtain the optimal stratification (i.e., zero variance) after one split.

In [ ]:
import logging

# Import the root package logger
strat_logger = logging.getLogger(adss.__name__)

# set the desired logging level
strat_logger.setLevel(logging.DEBUG)

# and add the corresponding hanlder andd formatter.
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.DEBUG)
c_formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
c_handler.setFormatter(c_formatter)
strat_logger.addHandler(c_handler)

# Using the one dimensional case, where we already have the optimal solution after one split.
dimension = 1

def test_fun(xi: np.ndarray):
    return np.sum(xi ** 2, axis=1) <= 0.5 ** 2

est_hyper = estimator(test_fun, dimension, 1000, SR_const, alpha, type_strat='hyperrect', rand_gen=None, dynamic=False)
QoI, ignore, N_strat, QoI_var = est_hyper.solve()
print(f'After splitting the domain into {N_strat} strata, we found that the quantity of interest is {QoI} with a variance of {QoI_var}')